In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [11]:
# 1. Load the data
csv_file = "final_processed_data.csv"
geojson_file = "Cleaned_Municipality.geojson"

In [6]:
# Load final_processed_data.csv
df = pd.read_csv(csv_file)

In [14]:
import json

with open("Cleaned_Municipality.geojson") as f:
    data = json.load(f)

In [16]:
import json

with open("Cleaned_Municipality.geojson") as f:
    data = json.load(f)

# Convert the list to a FeatureCollection
geojson_featurecollection = {
    "type": "FeatureCollection",
    "features": data
}

# Save the fixed GeoJSON
with open("Cleaned_Municipalities_fixed.geojson", "w") as f:
    json.dump(geojson_featurecollection, f)


In [17]:
# 2. Convert the CSV to a GeoDataFrame
geometry = [Point(xy) for xy in zip(df['Longitude'], df['Latitude'])]
gdf_points = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")  # CRS 4326 is for latitude/longitude


In [19]:
gdf_municipalities = gpd.GeoDataFrame.from_features(geojson_featurecollection["features"])


In [20]:
gdf_joined = gpd.sjoin(gdf_points, gdf_municipalities, how="left", predicate="within")


C:\Users\prova\AppData\Local\Temp\ipykernel_14056\1121728352.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  gdf_joined = gpd.sjoin(gdf_points, gdf_municipalities, how="left", predicate="within")


In [21]:
df['Municipality'] = gdf_joined['ref:ISTAT']


In [22]:
df.head()

,ID,Name,Type,Latitude,Longitude,Description,Details,Address,Geometry,Contact,Municipality
0,498740308,Bus del nono,caves,45.782194,10.992617,NaN,NaN,NaN,POINT (10.9926165 45.7821944),NaN,22001
1,508788420,Prigione del Sasso,caves,45.921416,10.888087,NaN,NaN,NaN,POINT (10.888087 45.9214156),NaN,22006
2,600025838,Grotta Damiano Chiesa,caves,45.853438,11.036204,NaN,NaN,NaN,POINT (11.0362043 45.8534376),NaN,22161
3,610991691,Unnamed,caves,45.881934,10.884507,NaN,NaN,NaN,POINT (10.8845066 45.8819343),NaN,22124
4,641260808,Miniera dell'Aubis,caves,46.111971,11.313204,NaN,NaN,NaN,POINT (11.3132041 46.1119712),NaN,22085


In [24]:
df.Municipality.isna().sum()/df.shape[0]

0.09287107531617693

In [25]:
df.shape

(31786, 11)

In [26]:
df.Municipality.isna().sum()

2952

In [27]:
df.to_csv("Tourist_Attractions_Final.csv", index=False)